In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from os.path import join

import tensorflow as tf
import keras
from keras import layers, models, optimizers
from keras.utils import to_categorical

C:\Users\KimJinSeop\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\KimJinSeop\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\KimJinSeop\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\KimJinSeop\Anaconda3\envs\tensorfl

In [2]:
datax = np.load(join('data','MNIST.npy'))
datay = np.load(join('data','Label.npy'))

In [3]:
datax_norm = datax/255
print('최대 : {}, 최소 : {}'.format(np.max(datax_norm), np.min(datax_norm)))

최대 : 1.0, 최소 : 0.0


In [4]:
datay_onehot = to_categorical(datay)
print(datay[0:10])
print(datay_onehot[0:10,:])

[5 0 4 1 9 2 1 3 1 4]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [5]:
from sklearn.model_selection import train_test_split
trnx, tstx, trny, tsty = train_test_split(datax_norm, datay_onehot, test_size=0.3, random_state=111)
print(trnx.shape)
print(tstx.shape)
print(trny.shape)
print(trnx[0].shape)

(42000, 28, 28)
(18000, 28, 28)
(42000, 10)
(28, 28)


In [6]:
input_shape = (28,28,1)

cnn_model = models.Sequential()

cnn_model.add(layers.Conv2D(8, (2,2), padding='same', input_shape=input_shape))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation("relu"))
cnn_model.add(layers.MaxPooling2D((2,2)))

cnn_model.add(layers.Conv2D(16, (2,2), padding='same', input_shape=input_shape))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation("relu"))
cnn_model.add(layers.MaxPooling2D((2,2)))

cnn_model.add(layers.Conv2D(16, (2,2), padding='same'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation("relu"))
cnn_model.add(layers.Dropout(0.2))
cnn_model.add(layers.MaxPooling2D((2,2)))

cnn_model.add(layers.Flatten())

cnn_model.add(layers.Dense(units = 64, activation = "relu"))
cnn_model.add(layers.Dense(units = 10, activation =  "softmax"))

cnn_model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 8)         40        
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 16)        528       
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 16)        64        
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 16)       

In [8]:
trnx_reshape = trnx.reshape(-1,28,28,1)
tstx_reshape = tstx.reshape(-1,28,28,1)
print(trnx_reshape.shape)

(42000, 28, 28, 1)


In [9]:
history = cnn_model.fit(trnx_reshape, trny, validation_data = [tstx_reshape, tsty], batch_size=512, epochs=50)

Instructions for updating:
Use tf.cast instead.
Train on 42000 samples, validate on 18000 samples
Epoch 1/50
42000/42000 [==============================] - 64s 2ms/step - loss: 1.2073 - accuracy: 0.6174 - val_loss: 1.8733 - val_accuracy: 0.2266
Epoch 2/50
42000/42000 [==============================] - 64s 2ms/step - loss: 0.3284 - accuracy: 0.9011 - val_loss: 1.5889 - val_accuracy: 0.3780
Epoch 3/50
42000/42000 [==============================] - 65s 2ms/step - loss: 0.2074 - accuracy: 0.9355 - val_loss: 0.9148 - val_accuracy: 0.6677
Epoch 4/50
42000/42000 [==============================] - 65s 2ms/step - loss: 0.1698 - accuracy: 0.9479 - val_loss: 0.3948 - val_accuracy: 0.8841
Epoch 5/50
42000/42000 [==============================] - 65s 2ms/step - loss: 0.1409 - accuracy: 0.9567 - val_loss: 0.1843 - val_accuracy: 0.9527
Epoch 6/50
42000/42000 [==============================] - 68s 2ms/step - loss: 0.1250 - accuracy: 0.9615 - val_loss: 0.1234 - val_accuracy: 0.9657
Epoch 7/50
42000/420